## Serving LLMs with MLflow: Leveraging Custom PyFunc


### Introduction

This tutorial guides you through saving and deploying Large Language Models (LLMs) using a custom `pyfunc` with MLflow, ideal for models not directly supported by MLflow's default transformers flavor.

### Learning Objectives

- Understand the need for custom `pyfunc` definitions in specific model scenarios.
- Learn to create a custom `pyfunc` to manage model dependencies and interface data.
- Gain insights into simplifying user interfaces in deployed environments with custom `pyfunc`.

#### The Challenge with Default Implementations
While MLflow's `transformers` flavor generally handles models from the HuggingFace Transformers library, some models or configurations might not align with this standard approach. In such cases, like ours, where the model cannot utilize the default `pipeline` type, we face a unique challenge of deploying these models using MLflow.

#### The Power of Custom PyFunc
To address this, MLflow's custom `pyfunc` comes to the rescue. It allows us to:

- Handle model loading and its dependencies efficiently.
- Customize the inference process to suit specific model requirements.
- Adapt interface data to create a user-friendly environment in deployed applications.

Our focus will be on the practical application of a custom `pyfunc` to deploy LLMs effectively within MLflow's ecosystem.

By the end of this tutorial, you'll be equipped with the knowledge to tackle similar challenges in your machine learning projects, leveraging the full potential of MLflow for custom model deployments.

### Important Considerations Before Proceeding

#### Hardware Recommendations
This guide demonstrates the usage of a particularly large and intricate Large Language Model (LLM). Given its complexity:

- **GPU Requirement**: It's **strongly advised** to run this example on a system with a CUDA-capable GPU that possesses at least 64GB of VRAM.
- **CPU Caution**: While technically feasible, executing the model on a CPU can result in extremely prolonged inference times, potentially taking tens of minutes for a single prediction, even on top-tier CPUs. The final cell of this notebook is deliberately not executed due to the limitations with performance when running this model on a CPU-only system. However, with an appropriately powerful GPU, the total runtime of this notebook is ~8 minutes end to end.

#### Execution Recommendations
If you're considering running the code in this notebook:

- **Performance**: For a smoother experience and to truly harness the model's capabilities, use hardware aligned with the model's design.

- **Dependencies**: Ensure you've installed the recommended dependencies for optimal model performance. These are crucial for efficient model loading, initialization, attention computations, and inference processing:

```bash
pip install xformers==0.0.20 einops==0.6.1 flash-attn==v1.0.3.post0 triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir#subdirectory=python
```

In [1]:
# Load necessary libraries

import accelerate
import torch
import transformers
from huggingface_hub import snapshot_download

import mlflow

/home/lwh/.conda/envs/mlflow-llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Downloading the Model and Tokenizer

First, we need to download our model and tokenizer. Here's how we do it:

In [2]:
# Download the MPT-7B instruct model and tokenizer to a local directory cache
# snapshot_location = snapshot_download(repo_id="mosaicml/mpt-7b-instruct", local_dir="mpt-7b")
snapshot_location = '/home/lwh/Documents/Code/MLflow-LLM/mpt-7b'

#### Defining the Custom PyFunc

Now, let's define our custom `pyfunc`. This will dictate how our model loads its dependencies and how it performs predictions. Notice how we've wrapped the intricacies of the model within this class.

In [3]:
class MPT(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        This method initializes the tokenizer and language model
        using the specified model snapshot directory.
        """
        # Initialize tokenizer and language model
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            context.artifacts["snapshot"], padding_side="left"
        )

        config = transformers.AutoConfig.from_pretrained(
            context.artifacts["snapshot"], trust_remote_code=True
        )
        # If you are running this in a system that has a sufficiently powerful GPU with available VRAM,
        # uncomment the configuration setting below to leverage triton.
        # Note that triton dramatically improves the inference speed performance

        # config.attn_config["attn_impl"] = "triton"

        self.model = transformers.AutoModelForCausalLM.from_pretrained(
            context.artifacts["snapshot"],
            config=config,
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
        )

        # NB: If you do not have a CUDA-capable device or have torch installed with CUDA support
        # this setting will not function correctly. Setting device to 'cpu' is valid, but
        # the performance will be very slow.
        # self.model.to(device="cpu")
        # If running on a GPU-compatible environment, uncomment the following line:
        self.model.to(device="cuda")

        self.model.eval()

    def _build_prompt(self, instruction):
        """
        This method generates the prompt for the model.
        """
        INSTRUCTION_KEY = "### Instruction:"
        RESPONSE_KEY = "### Response:"
        INTRO_BLURB = (
            "Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request."
        )

        return f"""{INTRO_BLURB}
        {INSTRUCTION_KEY}
        {instruction}
        {RESPONSE_KEY}

        """

    def predict(self, context, model_input, params=None):
        """
        This method generates prediction for the given input.
        """
        prompt = model_input["prompt"][0]

        # Retrieve or use default values for temperature and max_tokens
        temperature = params.get("temperature", 0.1) if params else 0.1
        max_tokens = params.get("max_tokens", 1000) if params else 1000

        # Build the prompt
        prompt = self._build_prompt(prompt)

        # Encode the input and generate prediction
        # NB: Sending the tokenized inputs to the GPU here explicitly will not work if your system does not have CUDA support.
        # If attempting to run this with GPU support, change 'cpu' to 'cuda' for maximum performance
        encoded_input = self.tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        output = self.model.generate(
            encoded_input,
            do_sample=True,
            temperature=temperature,
            max_new_tokens=max_tokens,
        )

        # Removing the prompt from the generated text
        prompt_length = len(self.tokenizer.encode(prompt, return_tensors="pt")[0])
        generated_response = self.tokenizer.decode(
            output[0][prompt_length:], skip_special_tokens=True
        )

        return {"candidates": [generated_response]}

### Building the Prompt

One key aspect of our custom `pyfunc` is the construction of a model prompt. Instead of the end-user having to understand and construct this prompt, our custom `pyfunc` takes care of it. This ensures that regardless of the intricacies of the model's requirements, the end-user interface remains simple and consistent.

Review the method `_build_prompt()` inside our class above to see how custom input processing logic can be added to a custom pyfunc to support required translations of user-input data into a format that is compatible with the wrapped model instance. 


In [5]:
import numpy as np
import pandas as pd

import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types import ColSpec, DataType, ParamSchema, ParamSpec, Schema

# Define input and output schema
input_schema = Schema(
    [
        ColSpec(DataType.string, "prompt"),
    ]
)
output_schema = Schema([ColSpec(DataType.string, "candidates")])

parameters = ParamSchema(
    [
        ParamSpec("temperature", DataType.float, np.float32(0.1), None),
        ParamSpec("max_tokens", DataType.integer, np.int32(1000), None),
    ]
)

signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=parameters)


# Define input example
input_example = pd.DataFrame({"prompt": ["What is Reinfoncement learning?"]})

#### Set the experiment that we're going to be logging our custom model to

If the experiment doesn't already exist, MLflow will create a new experiment with this name and will alert you that it has created a new experiment.

In [6]:
# If you are running this tutorial in local mode, leave the next line commented out.
# Otherwise, uncomment the following line and set your tracking uri to your local or remote tracking server.

# mlflow.set_tracking_uri("http://127.0.0.1:8080")

mlflow.set_experiment(experiment_name="mpt-7b-instruct-evaluation")

<Experiment: artifact_location='file:///home/lwh/Documents/Code/MLflow-LLM/mlruns/566831076974487757', creation_time=1721702364783, experiment_id='566831076974487757', last_update_time=1721702364783, lifecycle_stage='active', name='mpt-7b-instruct-evaluation', tags={}>

In [7]:
# Get the current base version of torch that is installed, without specific version modifiers
torch_version = torch.__version__.split("+")[0]

# Start an MLflow run context and log the MPT-7B model wrapper along with the param-included signature to
# allow for overriding parameters at inference time
with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        "mpt-7b-instruct",
        python_model=MPT(),
        # NOTE: the artifacts dictionary mapping is critical! This dict is used by the load_context() method in our MPT() class.
        artifacts={"snapshot": snapshot_location},
        pip_requirements=[
            f"torch=={torch_version}",
            f"transformers=={transformers.__version__}",
            f"accelerate=={accelerate.__version__}",
            "einops",
            "sentencepiece",
        ],
        input_example=input_example,
        signature=signature,
    )

2024/07/23 11:03:53 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - sentencepiece (current: uninstalled, required: sentencepiece)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


#### Load the saved model

In [8]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

2024/07/23 11:04:05 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - sentencepiece (current: uninstalled, required: sentencepiece)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/lwh/.cache/huggingface/modules/transformers_modules/mpt-7b/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/home/lwh/.cache/huggingface/modules/transformers_modules/mpt-7b/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(Us

#### Test the model for inference

In [10]:
# The execution of this is commented out for the purposes of runtime on CPU.
# If you are running this on a system with a sufficiently powerful GPU, you may uncomment and interface with the model!

loaded_model.predict(pd.DataFrame(
    {"prompt": ["What is machine learning?"]}), params={"temperature": 0.6, "max_tokens": 1000}
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


{'candidates': ['Machine learning is a field of study that uses algorithms to learn from data. Machine learning algorithms are mostly based on statistical learning methods. Machine learning algorithms are used for a variety of tasks, including pattern recognition, text classification, speech recognition, and computer vision. Machine learning is often used in conjunction with data mining and artificial intelligence.\n        \n        In simple terms, machine learning algorithms learn from the data and improve over time. For example, the Google search algorithm uses machine learning to improve its search results for you over time.']}

In [9]:
loaded_model.predict(pd.DataFrame(
    {"prompt": ["What is Reinfoncement learning?"]}), params={"temperature": 0.6, "max_tokens": 1000}
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/lwh/.cache/huggingface/modules/transformers_modules/mpt-7b/attention.py:89: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating 

{'candidates': ['Reinforcement learning (RL) is an area of machine learning concerned with how agents maximize a reward in an environment. More specifically, it is a technique for learning the best action to take in a given situation, through trial and error. The agent tries out actions and receives feedback, either positive (reward) or negative (penalty), that tells it which actions are better or worse.\n\n        Reinforcement learning is a field of study that focuses on how to teach artificial intelligence to learn through trial and error. It is based on the idea of what is known as the "reinforcement signal." The agent tries a certain action and receives feedback about how well that action worked. This feedback is then used to improve the agent\'s behavior.\n\n        In machine learning, the concept of reinforcement learning is used to describe a class of algorithms that learn through trial and error. The agent performs an action, receives feedback about the action, and uses this 

### Conclusion

Through this tutorial, we've seen the power and flexibility of MLflow's custom `pyfunc`. By understanding the specific needs of our model and defining a custom `pyfunc` to cater to those needs, we can ensure a seamless deployment process and a user-friendly interface.
